# Filtering the data to use in forecasting solar flares
***


In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Load dataframes
df_sharp = pd.read_csv("./sharp_data/sharp_data.csv")
df_goes = pd.read_csv("./goes_data/goes_all_probes.csv")
df_solar = pd.read_csv("./other_data/daily_solar_data.csv")
#df_noaa_radio = pd.read_csv("./other_data/noaa_radio_flux.csv")
#df_penticton_radio = pd.read_csv("./other_data/penticton_radio_flux.csv")
#df_xrt = pd.read_csv("./other_data/xrt_flarecat.csv")

# SHARP dataset
***

In [3]:
print(df_sharp.info())
print(df_sharp.shape)
print(df_sharp.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476418 entries, 0 to 4476417
Data columns (total 21 columns):
 #   Column    Dtype  
---  ------    -----  
 0   DATE-OBS  object 
 1   T_OBS     object 
 2   T_REC     object 
 3   USFLUX    float64
 4   MEANGAM   float64
 5   MEANGBT   float64
 6   MEANGBZ   float64
 7   MEANGBH   float64
 8   MEANJZD   float64
 9   TOTUSJZ   float64
 10  MEANALP   float64
 11  MEANJZH   float64
 12  TOTUSJH   float64
 13  ABSNJZH   float64
 14  SAVNCPP   float64
 15  MEANPOT   float64
 16  TOTPOT    float64
 17  MEANSHR   float64
 18  SHRGT45   float64
 19  R_VALUE   float64
 20  HARPNUM   int64  
dtypes: float64(17), int64(1), object(3)
memory usage: 717.2+ MB
None
(4476418, 21)
                  DATE-OBS                    T_OBS                    T_REC  \
0  2010-04-30T23:58:22.20Z  2010.05.01_00:00:04_TAI  2010.05.01_00:00:00_TAI   
1  2010-05-01T00:10:22.20Z  2010.05.01_00:12:04_TAI  2010.05.01_00:12:00_TAI   
2  2010-05-01T00:22:22.20Z  2010.0

In [4]:
# Function to convert TAI to UTC
def tai_to_utc(dt):
    # dt is naive datetime (parsed from _TAI)
    if dt < pd.Timestamp("2012-07-01"):
        offset = 34
    elif dt < pd.Timestamp("2015-07-01"):
        offset = 35
    elif dt < pd.Timestamp("2017-01-01"):
        offset = 36
    else:
        offset = 37
    return dt - pd.Timedelta(seconds=offset)

In [5]:
# Convert the time labels to datetime objects
df_sharp["DATE-OBS_time"] = pd.to_datetime(df_sharp["DATE-OBS"], format="%Y-%m-%dT%H:%M:%S.%fZ", utc=True)

df_sharp["T_OBS_time"] = pd.to_datetime(df_sharp["T_OBS"].str.replace("_TAI",""), format="%Y.%m.%d_%H:%M:%S")
df_sharp["T_REC_time"] = pd.to_datetime(df_sharp["T_REC"].str.replace("_TAI",""), format="%Y.%m.%d_%H:%M:%S")

df_sharp["T_OBS_time"] = df_sharp["T_OBS_time"].apply(tai_to_utc).dt.tz_localize("UTC")
df_sharp["T_REC_time"] = df_sharp["T_REC_time"].apply(tai_to_utc).dt.tz_localize("UTC")

print(df_sharp.head())

                  DATE-OBS                    T_OBS                    T_REC  \
0  2010-04-30T23:58:22.20Z  2010.05.01_00:00:04_TAI  2010.05.01_00:00:00_TAI   
1  2010-05-01T00:10:22.20Z  2010.05.01_00:12:04_TAI  2010.05.01_00:12:00_TAI   
2  2010-05-01T00:22:22.20Z  2010.05.01_00:24:04_TAI  2010.05.01_00:24:00_TAI   
3  2010-05-01T00:34:22.20Z  2010.05.01_00:36:04_TAI  2010.05.01_00:36:00_TAI   
4  2010-05-01T00:46:22.20Z  2010.05.01_00:48:04_TAI  2010.05.01_00:48:00_TAI   

         USFLUX  MEANGAM  MEANGBT  MEANGBZ  MEANGBH   MEANJZD       TOTUSJZ  \
0  6.510776e+21   28.337   66.808   84.497   32.193 -0.131873  5.777592e+12   
1  6.521054e+21   29.678   68.349   90.781   32.345 -0.113589  5.654726e+12   
2  6.917875e+21   28.441   67.682   89.127   32.411  0.061197  6.488687e+12   
3  6.973706e+21   28.031   67.166   85.321   31.966  0.053302  6.193157e+12   
4  7.228647e+21   26.980   64.805   76.349   32.647  0.011571  5.797055e+12   

   ...       SAVNCPP   MEANPOT        TOTPOT

In [ ]:
# Create a new data with only day for the aggeregation
# floor to day
df_sharp["DATE"] = df_sharp["DATE-OBS_time"].dt.floor("D")

# Many measureaments start just before midnight of the last day, but should be counted as part of the next day
# if very close to midnight, push to next day
# e.g. within last 5 minutes of the day
mask = df_sharp["DATE-OBS_time"].dt.hour == 23
mask &= df_sharp["DATE-OBS_time"].dt.minute >= 55
df_sharp.loc[mask, "DATE"] += pd.Timedelta(days=1)

print(df_sharp.head())

                  DATE-OBS                    T_OBS                    T_REC  \
0  2010-04-30T23:58:22.20Z  2010.05.01_00:00:04_TAI  2010.05.01_00:00:00_TAI   
1  2010-05-01T00:10:22.20Z  2010.05.01_00:12:04_TAI  2010.05.01_00:12:00_TAI   
2  2010-05-01T00:22:22.20Z  2010.05.01_00:24:04_TAI  2010.05.01_00:24:00_TAI   
3  2010-05-01T00:34:22.20Z  2010.05.01_00:36:04_TAI  2010.05.01_00:36:00_TAI   
4  2010-05-01T00:46:22.20Z  2010.05.01_00:48:04_TAI  2010.05.01_00:48:00_TAI   

         USFLUX  MEANGAM  MEANGBT  MEANGBZ  MEANGBH   MEANJZD       TOTUSJZ  \
0  6.510776e+21   28.337   66.808   84.497   32.193 -0.131873  5.777592e+12   
1  6.521054e+21   29.678   68.349   90.781   32.345 -0.113589  5.654726e+12   
2  6.917875e+21   28.441   67.682   89.127   32.411  0.061197  6.488687e+12   
3  6.973706e+21   28.031   67.166   85.321   31.966  0.053302  6.193157e+12   
4  7.228647e+21   26.980   64.805   76.349   32.647  0.011571  5.797055e+12   

   ...   MEANPOT        TOTPOT  MEANSHR  SHR

In [7]:
# search for NaNs

# count NaNs per column
nan_count = df_sharp.isna().sum()
print(nan_count)

# fraction between 0 and 1
nan_fraction = df_sharp.isna().mean()
print(nan_fraction)

DATE-OBS             0
T_OBS                0
T_REC                0
USFLUX               0
MEANGAM          44216
MEANGBT          43413
MEANGBZ          43399
MEANGBH          43399
MEANJZD          43399
TOTUSJZ              0
MEANALP          43413
MEANJZH          43404
TOTUSJH              0
ABSNJZH              0
SAVNCPP              0
MEANPOT          43091
TOTPOT               0
MEANSHR          48665
SHRGT45          43091
R_VALUE           4318
HARPNUM              0
DATE-OBS_time        0
T_OBS_time           0
T_REC_time           0
DATE                 0
dtype: int64
DATE-OBS         0.000000
T_OBS            0.000000
T_REC            0.000000
USFLUX           0.000000
MEANGAM          0.009878
MEANGBT          0.009698
MEANGBZ          0.009695
MEANGBH          0.009695
MEANJZD          0.009695
TOTUSJZ          0.000000
MEANALP          0.009698
MEANJZH          0.009696
TOTUSJH          0.000000
ABSNJZH          0.000000
SAVNCPP          0.000000
MEANPOT          0.009

In [8]:
# Search for infinities
print(np.isinf(df_sharp.loc[:, 'USFLUX':'HARPNUM']).sum().sum())

50


In [9]:
# Replace infinite values with NaN
df_sharp = df_sharp.replace([np.inf, -np.inf], np.nan)

In [10]:
# count NaNs per HARPNUM per day
nan_counts = (
    df_sharp.groupby(["HARPNUM", "DATE"])
      .apply(lambda g: g.isna().sum(), include_groups=False)
)
print(nan_counts)

# also total rows, to compute fractions
nan_fractions = (
    df_sharp.groupby(["HARPNUM", "DATE"])
      .apply(lambda g: g.isna().mean(), include_groups=False)
)
print(nan_fractions)

                                   DATE-OBS  T_OBS  T_REC  USFLUX  MEANGAM  \
HARPNUM DATE                                                                 
1       2010-05-01 00:00:00+00:00         0      0      0       0        0   
        2010-05-02 00:00:00+00:00         0      0      0       0        0   
        2010-05-03 00:00:00+00:00         0      0      0       0        0   
        2010-05-04 00:00:00+00:00         0      0      0       0        0   
        2010-05-05 00:00:00+00:00         0      0      0       0        0   
...                                     ...    ...    ...     ...      ...   
12519   2025-01-01 00:00:00+00:00         0      0      0       0        0   
12523   2024-12-31 00:00:00+00:00         0      0      0       0        0   
        2025-01-01 00:00:00+00:00         0      0      0       0        0   
12527   2024-12-31 00:00:00+00:00         0      0      0       0        0   
        2025-01-01 00:00:00+00:00         0      0      0       

In [11]:
# check if NaNs are correlated with HARPNUM or DATE
nan_fractions = nan_fractions.reset_index()
summary = nan_fractions.melt(id_vars=["HARPNUM", "DATE"], 
                             var_name="column", value_name="nan_fraction")

# filter only cases with >0
summary = summary[summary["nan_fraction"] > 0].sort_values("nan_fraction", ascending=False)

print(summary.head(100))

        HARPNUM                      DATE   column  nan_fraction
727831     6841 2016-11-12 00:00:00+00:00  MEANPOT           1.0
524725     2541 2013-03-07 00:00:00+00:00  MEANJZH           1.0
290427     2464 2013-02-09 00:00:00+00:00  MEANGBZ           1.0
524720     2533 2013-03-18 00:00:00+00:00  MEANJZH           1.0
524721     2537 2013-03-06 00:00:00+00:00  MEANJZH           1.0
...         ...                       ...      ...           ...
524494     2488 2013-02-17 00:00:00+00:00  MEANJZH           1.0
524508     2491 2013-02-15 00:00:00+00:00  MEANJZH           1.0
524486     2482 2013-02-11 00:00:00+00:00  MEANJZH           1.0
524485     2481 2013-02-16 00:00:00+00:00  MEANJZH           1.0
852724     2642 2013-04-05 00:00:00+00:00  SHRGT45           1.0

[100 rows x 4 columns]


In [12]:
# NaNs are not correlated with HARPNUM or DATE. Entire days or HARPNUMs are not missing.
# We can delete rows with NaNs without introducing bias.

df_sharp_clean = df_sharp.dropna()

In [13]:
# average over days and spots

# 1. daily average per spot
df_daily_per_spot = (
    df_sharp_clean.groupby(["HARPNUM", "DATE"])
      .mean(numeric_only=True)
      .reset_index()
)
print(df_daily_per_spot.head())

# 2. daily average across spots (ignore HARPNUM)
df_daily_all_spots = (
    df_sharp_clean.groupby("DATE")
      .mean(numeric_only=True)
      .reset_index()
)
print(df_daily_all_spots.head())

   HARPNUM                      DATE        USFLUX    MEANGAM     MEANGBT  \
0        1 2010-05-01 00:00:00+00:00  6.703258e+21  25.728111   72.767846   
1        1 2010-05-02 00:00:00+00:00  9.855321e+21  25.814383   79.668617   
2        1 2010-05-03 00:00:00+00:00  9.420379e+21  28.120769   94.159909   
3        1 2010-05-04 00:00:00+00:00  6.441307e+21  29.274752  111.013884   
4        1 2010-05-05 00:00:00+00:00  5.100163e+21  30.340793  120.692843   

      MEANGBZ    MEANGBH   MEANJZD       TOTUSJZ   MEANALP   MEANJZH  \
0   82.501701  32.511701 -0.091966  6.021028e+12 -0.001429 -0.000457   
1   81.215375  33.471042 -0.181589  8.825466e+12  0.000624  0.000202   
2   94.493545  37.063504 -0.002756  9.878426e+12  0.007282  0.002162   
3  110.362157  43.795851  0.243887  7.983555e+12  0.002613  0.000733   
4  119.220595  48.880603  0.358636  7.487984e+12 -0.004161 -0.001062   

      TOTUSJH    ABSNJZH       SAVNCPP      MEANPOT        TOTPOT    MEANSHR  \
0  293.488923  19.927444

# GOES dataset
***

In [14]:
print(df_goes.info())
print(df_goes.shape)
print(df_goes.head())
print(df_goes.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12911040 entries, 0 to 12911039
Data columns (total 66 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   datetime                         object 
 1   g10_xrs_xs                       float64
 2   g10_xrs_xl                       float64
 3   g11_xrs_xs                       float64
 4   g11_xrs_xl                       float64
 5   g12_xrs_xs                       float64
 6   g12_xrs_xl                       float64
 7   g13_xrs_A_QUAL_FLAG              float64
 8   g13_xrs_A_NUM_PTS                float64
 9   g13_xrs_A_AVG                    float64
 10  g13_xrs_B_QUAL_FLAG              float64
 11  g13_xrs_B_NUM_PTS                float64
 12  g13_xrs_B_AVG                    float64
 13  g14_xrs_A_QUAL_FLAG              float64
 14  g14_xrs_A_NUM_PTS                float64
 15  g14_xrs_A_AVG                    float64
 16  g14_xrs_B_QUAL_FLAG              float64
 17  g14_xr

In [15]:
# Convert the time labels to datetime objects
df_goes["datetime_time"] = pd.to_datetime(df_goes["datetime"], utc=True)

# Create a new data with only day for the aggeregation
# floor to day
df_goes["DATE"] = df_goes["datetime_time"].dt.floor("D")

In [16]:
# some values are not NaN but are invalid, e.g. -1 or 9999
for col in df_goes.columns:
    if df_goes[col].dtype in [float, int]:
        df_goes.loc[df_goes[col] < 0, col] = np.nan

In [17]:
# Search for infinities
print(np.isinf(df_goes.loc[:, 'g10_xrs_xs':'g15_xrs_B_AVG']).sum().sum())

0


In [18]:
# Replace infinite values with NaN
df_goes = df_goes.replace([np.inf, -np.inf], np.nan)

In [19]:
# count NaNs per column
nan_count = df_goes.isna().sum()
print(nan_count)

# fraction between 0 and 1
nan_fraction = df_goes.isna().mean()
print(nan_fraction)

datetime                             0
g10_xrs_xs                     7259222
g10_xrs_xl                     7259253
g11_xrs_xs                    12018767
g11_xrs_xl                    12018781
                                ...   
g17_xrs_xrsb_flux_observed    11363142
g17_xrs_xrsb_num              11306880
g17_xrs_yaw_flip_flag         11306880
datetime_time                        0
DATE                                 0
Length: 68, dtype: int64
datetime                      0.000000
g10_xrs_xs                    0.562249
g10_xrs_xl                    0.562252
g11_xrs_xs                    0.930891
g11_xrs_xl                    0.930892
                                ...   
g17_xrs_xrsb_flux_observed    0.880111
g17_xrs_xrsb_num              0.875753
g17_xrs_yaw_flip_flag         0.875753
datetime_time                 0.000000
DATE                          0.000000
Length: 68, dtype: float64


In [20]:
# count NaNs per day
nan_counts = (
    df_goes.groupby(["DATE"])
      .apply(lambda g: g.isna().sum(), include_groups=False)
)
print(nan_counts)

# also total rows, to compute fractions
nan_fractions = (
    df_goes.groupby(["DATE"])
      .apply(lambda g: g.isna().mean(), include_groups=False)
)
print(nan_fractions)

                           datetime  g10_xrs_xs  g10_xrs_xl  g11_xrs_xs  \
DATE                                                                      
1998-07-01 00:00:00+00:00         0           0           0        1440   
1998-07-02 00:00:00+00:00         0           1           1        1440   
1998-07-03 00:00:00+00:00         0         121         121        1440   
1998-07-04 00:00:00+00:00         0           0           0        1440   
1998-07-05 00:00:00+00:00         0           0           0        1440   
...                             ...         ...         ...         ...   
2024-08-07 00:00:00+00:00         0        1440        1440        1440   
2024-08-08 00:00:00+00:00         0        1440        1440        1440   
2024-08-09 00:00:00+00:00         0        1440        1440        1440   
2024-08-10 00:00:00+00:00         0        1440        1440        1440   
2024-08-11 00:00:00+00:00         0        1440        1440        1440   

                        

In [21]:
# Sometimes, entire days are missing. Better not to filter them out. Better to interpolate them.
# First, later GOES satellites have better coverage, so we can use them first and fill in missing data using earlier satellites only when necessary.

# define priority list: latest satellite first
xrs_A_cols = ['g15_xrs_A_AVG','g14_xrs_A_AVG','g13_xrs_A_AVG', 'g12_xrs_xs', 'g11_xrs_xs', 'g10_xrs_xs']
xrs_B_cols = ['g15_xrs_B_AVG','g14_xrs_B_AVG','g13_xrs_B_AVG', 'g12_xrs_xl', 'g11_xrs_xl', 'g10_xrs_xl']

# create a single column per channel by filling NaNs from newest to oldest
df_goes['xrs_A'] = df_goes[xrs_A_cols].bfill(axis=1).iloc[:,0]
df_goes['xrs_B'] = df_goes[xrs_B_cols].bfill(axis=1).iloc[:,0]

print(df_goes.head())

              datetime    g10_xrs_xs    g10_xrs_xl  g11_xrs_xs  g11_xrs_xl  \
0  1998-07-01 00:00:00  3.350000e-09  1.270000e-07         NaN         NaN   
1  1998-07-01 00:01:00  3.330000e-09  1.270000e-07         NaN         NaN   
2  1998-07-01 00:02:00  3.330000e-09  1.270000e-07         NaN         NaN   
3  1998-07-01 00:03:00  3.330000e-09  1.270000e-07         NaN         NaN   
4  1998-07-01 00:04:00  3.330000e-09  1.270000e-07         NaN         NaN   

   g12_xrs_xs  g12_xrs_xl  g13_xrs_A_QUAL_FLAG  g13_xrs_A_NUM_PTS  \
0         NaN         NaN                  NaN                NaN   
1         NaN         NaN                  NaN                NaN   
2         NaN         NaN                  NaN                NaN   
3         NaN         NaN                  NaN                NaN   
4         NaN         NaN                  NaN                NaN   

   g13_xrs_A_AVG  ...  g17_xrs_xrsb_flag_excluded  g17_xrs_xrsb_flux  \
0            NaN  ...                       

In [ ]:
# Interpolating on the short and long x-rays channels

# set datetime as index (required for method='time')
df_goes = df_goes.set_index('DATE')

# interpolate only selected columns
df_goes[['xrs_A', 'xrs_B']] = df_goes[['xrs_A', 'xrs_B']].interpolate(method='time')

# reset index
df_goes = df_goes.reset_index()

print(df_goes.head())

                       DATE             datetime    g10_xrs_xs    g10_xrs_xl  \
0 1998-07-01 00:00:00+00:00  1998-07-01 00:00:00  3.350000e-09  1.270000e-07   
1 1998-07-01 00:00:00+00:00  1998-07-01 00:01:00  3.330000e-09  1.270000e-07   
2 1998-07-01 00:00:00+00:00  1998-07-01 00:02:00  3.330000e-09  1.270000e-07   
3 1998-07-01 00:00:00+00:00  1998-07-01 00:03:00  3.330000e-09  1.270000e-07   
4 1998-07-01 00:00:00+00:00  1998-07-01 00:04:00  3.330000e-09  1.270000e-07   

   g11_xrs_xs  g11_xrs_xl  g12_xrs_xs  g12_xrs_xl  g13_xrs_A_QUAL_FLAG  \
0         NaN         NaN         NaN         NaN                  NaN   
1         NaN         NaN         NaN         NaN                  NaN   
2         NaN         NaN         NaN         NaN                  NaN   
3         NaN         NaN         NaN         NaN                  NaN   
4         NaN         NaN         NaN         NaN                  NaN   

   g13_xrs_A_NUM_PTS  ...  g17_xrs_xrsb_flag  g17_xrs_xrsb_flag_excluded  

In [23]:
# Average over days and keeping also min and max of the day
df_daily_goes = (
    df_goes.groupby("DATE")
      .agg(
          xrs_A_mean=('xrs_A', 'mean'),
          xrs_A_min=('xrs_A', 'min'),
          xrs_A_max=('xrs_A', 'max'),
          xrs_B_mean=('xrs_B', 'mean'),
          xrs_B_min=('xrs_B', 'min'),
          xrs_B_max=('xrs_B', 'max'),
      )
      .reset_index()
)

print(df_daily_goes.head())

                       DATE    xrs_A_mean     xrs_A_min     xrs_A_max  \
0 1998-07-01 00:00:00+00:00  5.222819e-09  1.450000e-09  1.410000e-07   
1 1998-07-02 00:00:00+00:00  8.806660e-09  1.570000e-09  4.510000e-08   
2 1998-07-03 00:00:00+00:00  1.573242e-08  1.470000e-09  5.210000e-07   
3 1998-07-04 00:00:00+00:00  6.156368e-09  1.440000e-09  1.380000e-07   
4 1998-07-05 00:00:00+00:00  2.525917e-08  1.450000e-09  7.450000e-07   

     xrs_B_mean     xrs_B_min     xrs_B_max  
0  2.292431e-07  1.270000e-07  1.370000e-06  
1  4.277694e-07  2.140000e-07  8.990000e-07  
2  5.068799e-07  2.370000e-07  5.780000e-06  
3  3.646153e-07  2.480000e-07  1.430000e-06  
4  5.796479e-07  2.280000e-07  7.660000e-06  


# Solar data
***

In [153]:
print(df_solar.info())
print(df_solar.shape)
print(df_solar.head())
print(df_solar.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10330 entries, 0 to 10329
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Date                                10330 non-null  object
 1   Radio Flux 10.7cm                   10330 non-null  int64 
 2   Sunspot Number                      10330 non-null  int64 
 3   Sunspot Area (10^6 Hemis.)          10330 non-null  int64 
 4   New Regions                         10330 non-null  int64 
 5   Stanford Mean Solar Field (GOES15)  10330 non-null  object
 6   Stanford Background X-Ray Flux      10330 non-null  object
 7   Flares: C                           10330 non-null  int64 
 8   Flares: M                           10330 non-null  int64 
 9   Flares: X                           10330 non-null  int64 
 10  Flares: S                           10330 non-null  int64 
 11  Flares: 1                           10330 non-null  in

In [154]:
# Convert the time labels to datetime objects
df_solar["DATE"] = pd.to_datetime(df_solar["Date"], utc=True)

print(df_solar.head())

         Date  Radio Flux 10.7cm  Sunspot Number  Sunspot Area (10^6 Hemis.)  \
0  1997-01-01                 72               0                           0   
1  1997-01-02                 72               0                           0   
2  1997-01-03                 73               0                           0   
3  1997-01-04                 74              13                          10   
4  1997-01-05                 74              15                          20   

   New Regions Stanford Mean Solar Field (GOES15)  \
0            0                                  *   
1            0                                  *   
2            0                                  3   
3            1                                  *   
4            0                                  5   

  Stanford Background X-Ray Flux  Flares: C  Flares: M  Flares: X  Flares: S  \
0                           A0.5          0          0          0         -1   
1                           A0.5        

In [155]:
# clear up null values
df_solar.replace('*', np.nan, inplace=True)
df_solar.replace(-1, np.nan, inplace=True)
df_solar.replace('-999', np.nan, inplace=True)

print(df_solar.head())

         Date  Radio Flux 10.7cm  Sunspot Number  Sunspot Area (10^6 Hemis.)  \
0  1997-01-01                 72               0                           0   
1  1997-01-02                 72               0                           0   
2  1997-01-03                 73               0                           0   
3  1997-01-04                 74              13                          10   
4  1997-01-05                 74              15                          20   

   New Regions Stanford Mean Solar Field (GOES15)  \
0            0                                NaN   
1            0                                NaN   
2            0                                  3   
3            1                                NaN   
4            0                                  5   

  Stanford Background X-Ray Flux  Flares: C  Flares: M  Flares: X  Flares: S  \
0                           A0.5        0.0        0.0        0.0        NaN   
1                           A0.5        

In [ ]:
# Include label for flares class M or X
df_solar['flare_today'] = ((df_solar['Flares: M'] > 0) | (df_solar['Flares: X'] > 0)).astype(int)

# selecting years from 2000 to now
df_solar = df_solar[(df_solar['DATE'].dt.year >= 2000)]

print(df_solar.head())

# check on how many days there were flares of class M or X
print(df_solar['flare_today'].value_counts()/len(df_solar))

            Date  Radio Flux 10.7cm  Sunspot Number  \
1095  2000-01-01                130              69   
1096  2000-01-02                133              69   
1097  2000-01-03                133              77   
1098  2000-01-04                135             102   
1099  2000-01-05                137             100   

      Sunspot Area (10^6 Hemis.)  New Regions  \
1095                         540            0   
1096                         460            0   
1097                         480            0   
1098                         460            2   
1099                         410            1   

     Stanford Mean Solar Field (GOES15) Stanford Background X-Ray Flux  \
1095                                 89                           B5.7   
1096                                 79                           B3.7   
1097                                 30                           B3.6   
1098                                NaN                           B2.8   
1099

In [157]:
# checking for NaNs
nan_count = df_solar.isna().sum()
print(nan_count)

Date                                     0
Radio Flux 10.7cm                        0
Sunspot Number                           0
Sunspot Area (10^6 Hemis.)               0
New Regions                              0
Stanford Mean Solar Field (GOES15)    9049
Stanford Background X-Ray Flux        1433
Flares: C                                1
Flares: M                                1
Flares: X                                1
Flares: S                                0
Flares: 1                                0
Flares: 2                                0
Flares: 3                                0
DATE                                     0
flare_today                              0
dtype: int64


# Creating final dataframe
***

In [177]:
# Some dates are misisng, so bettee to create a full date range and merge the dataframes later

# Generate full date range
date_range = pd.date_range(
    start="2010-01-01", 
    end="2024-12-31", 
    freq="D", 
    tz="UTC"
)

# Create DataFrame
df_dates = pd.DataFrame({"DATE": date_range})

print(df_dates.head())
print(df_dates.tail())
print(len(df_dates))

                       DATE
0 2010-01-01 00:00:00+00:00
1 2010-01-02 00:00:00+00:00
2 2010-01-03 00:00:00+00:00
3 2010-01-04 00:00:00+00:00
4 2010-01-05 00:00:00+00:00
                          DATE
5474 2024-12-27 00:00:00+00:00
5475 2024-12-28 00:00:00+00:00
5476 2024-12-29 00:00:00+00:00
5477 2024-12-30 00:00:00+00:00
5478 2024-12-31 00:00:00+00:00
5479


In [178]:
# Merge dataframes

# merge DATA dataframe
df_merged = df_dates.merge(
    df_daily_all_spots, on='DATE', how='left')

# merge GOES data
df_merged = df_merged.merge(
    df_daily_goes, on='DATE', how='left')

# merge solar indices
df_merged = df_merged.merge(
    df_solar, on='DATE', how='left')

print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5936 entries, 0 to 5935
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   DATE                                5936 non-null   datetime64[ns, UTC]
 1   USFLUX                              5315 non-null   float64            
 2   MEANGAM                             5315 non-null   float64            
 3   MEANGBT                             5315 non-null   float64            
 4   MEANGBZ                             5315 non-null   float64            
 5   MEANGBH                             5315 non-null   float64            
 6   MEANJZD                             5315 non-null   float64            
 7   TOTUSJZ                             5315 non-null   float64            
 8   MEANALP                             5315 non-null   float64            
 9   MEANJZH                             5315 

In [179]:
# Check columns
print(df_merged.columns)

Index(['DATE', 'USFLUX', 'MEANGAM', 'MEANGBT', 'MEANGBZ', 'MEANGBH', 'MEANJZD',
       'TOTUSJZ', 'MEANALP', 'MEANJZH', 'TOTUSJH', 'ABSNJZH', 'SAVNCPP',
       'MEANPOT', 'TOTPOT', 'MEANSHR', 'SHRGT45', 'R_VALUE', 'HARPNUM',
       'xrs_A_mean', 'xrs_A_min', 'xrs_A_max', 'xrs_B_mean', 'xrs_B_min',
       'xrs_B_max', 'Date', 'Radio Flux 10.7cm', 'Sunspot Number',
       'Sunspot Area (10^6 Hemis.)', 'New Regions',
       'Stanford Mean Solar Field (GOES15)', 'Stanford Background X-Ray Flux',
       'Flares: C', 'Flares: M', 'Flares: X', 'Flares: S', 'Flares: 1',
       'Flares: 2', 'Flares: 3', 'flare_today'],
      dtype='object')


In [180]:
columns = [
    'DATE', 
    'USFLUX', 
    'MEANGAM', 
    'MEANGBT', 
    'MEANGBZ', 
    'MEANGBH', 
    'MEANJZD',
    'TOTUSJZ', 
    'MEANALP', 
    'MEANJZH', 
    'TOTUSJH', 
    'ABSNJZH', 
    'SAVNCPP',
    'MEANPOT', 
    'TOTPOT', 
    'MEANSHR', 
    'SHRGT45', 
    'R_VALUE',
    'xrs_A_mean', 
    'xrs_A_min', 
    'xrs_A_max', 
    'xrs_B_mean', 
    'xrs_B_min',
    'xrs_B_max',
    'Radio Flux 10.7cm', 
    'Sunspot Number',
    'Sunspot Area (10^6 Hemis.)', 
    'New Regions',
    'Flares: C', 
    'Flares: M', 
    'Flares: X',
    'flare_today']

In [181]:
df_flares = df_merged[columns].copy()

# Drop duplicate rows if any
df_flares = df_flares.drop_duplicates(subset=['DATE'])

print(df_flares.info())
print(df_flares.head())
print(df_flares.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 5479 entries, 0 to 5935
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   DATE                        5479 non-null   datetime64[ns, UTC]
 1   USFLUX                      4954 non-null   float64            
 2   MEANGAM                     4954 non-null   float64            
 3   MEANGBT                     4954 non-null   float64            
 4   MEANGBZ                     4954 non-null   float64            
 5   MEANGBH                     4954 non-null   float64            
 6   MEANJZD                     4954 non-null   float64            
 7   TOTUSJZ                     4954 non-null   float64            
 8   MEANALP                     4954 non-null   float64            
 9   MEANJZH                     4954 non-null   float64            
 10  TOTUSJH                     4954 non-null   float64            
 

In [182]:
# check for maximum dates
print(min(df_daily_all_spots['DATE']))
print(max(df_solar['DATE']))

2010-05-01 00:00:00+00:00
2024-04-12 00:00:00+00:00


In [183]:
# select the right year range
mask_1 = df_flares['DATE'] >= min(df_daily_all_spots['DATE'])
mask_2 = df_flares['DATE'] <= max(df_solar['DATE'])
df_flares = df_flares[mask_1 & mask_2]

print(df_flares.info())
print(df_flares.head())

<class 'pandas.core.frame.DataFrame'>
Index: 5096 entries, 120 to 5672
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   DATE                        5096 non-null   datetime64[ns, UTC]
 1   USFLUX                      4691 non-null   float64            
 2   MEANGAM                     4691 non-null   float64            
 3   MEANGBT                     4691 non-null   float64            
 4   MEANGBZ                     4691 non-null   float64            
 5   MEANGBH                     4691 non-null   float64            
 6   MEANJZD                     4691 non-null   float64            
 7   TOTUSJZ                     4691 non-null   float64            
 8   MEANALP                     4691 non-null   float64            
 9   MEANJZH                     4691 non-null   float64            
 10  TOTUSJH                     4691 non-null   float64            

In [184]:
# check for NaNs
nan_count = df_flares.isna().sum()
print(nan_count)

DATE                            0
USFLUX                        405
MEANGAM                       405
MEANGBT                       405
MEANGBZ                       405
MEANGBH                       405
MEANJZD                       405
TOTUSJZ                       405
MEANALP                       405
MEANJZH                       405
TOTUSJH                       405
ABSNJZH                       405
SAVNCPP                       405
MEANPOT                       405
TOTPOT                        405
MEANSHR                       405
SHRGT45                       405
R_VALUE                       405
xrs_A_mean                    460
xrs_A_min                     460
xrs_A_max                     460
xrs_B_mean                    460
xrs_B_min                     460
xrs_B_max                     460
Radio Flux 10.7cm              91
Sunspot Number                 91
Sunspot Area (10^6 Hemis.)     91
New Regions                    91
Flares: C                      91
Flares: M     

In [174]:
# Cleaning NaNs by interpolation

# Those 354 days with NaNs in flares are lost in my datasets. I decided to keep them as NaN andcreate a missing column.
df_flares["flare_missing"] = df_flares["flare_today"].isna().astype(int)

# set datetime as index (required for method='time')
df_flares = df_flares.set_index('DATE')

# The other columns I will interpolate by time and by rolling average if too big.
columns_to_interpolate = [
    'USFLUX', 
    'MEANGAM', 
    'MEANGBT', 
    'MEANGBZ', 
    'MEANGBH', 
    'MEANJZD',
    'TOTUSJZ', 
    'MEANALP', 
    'MEANJZH', 
    'TOTUSJH', 
    'ABSNJZH', 
    'SAVNCPP',
    'MEANPOT', 
    'TOTPOT', 
    'MEANSHR', 
    'SHRGT45', 
    'R_VALUE',
    'xrs_A_mean', 
    'xrs_A_min', 
    'xrs_A_max', 
    'xrs_B_mean', 
    'xrs_B_min',
    'xrs_B_max',
    'Radio Flux 10.7cm', 
    'Sunspot Number',
    'Sunspot Area (10^6 Hemis.)']

df_flares[columns_to_interpolate] = (
    df_flares[columns_to_interpolate]
    .fillna(df_flares[columns_to_interpolate].rolling(7, min_periods=1, center=True).mean())  # rolling average as interpolation
    .interpolate(method="time")  # fallback to interpolate up to 3 consecutive days
)

# reset index
df_flares = df_flares.reset_index()

nan_count = df_flares.isna().sum()
print(nan_count)

print(df_flares.head())

DATE                           0
USFLUX                         0
MEANGAM                        0
MEANGBT                        0
MEANGBZ                        0
MEANGBH                        0
MEANJZD                        0
TOTUSJZ                        0
MEANALP                        0
MEANJZH                        0
TOTUSJH                        0
ABSNJZH                        0
SAVNCPP                        0
MEANPOT                        0
TOTPOT                         0
MEANSHR                        0
SHRGT45                        0
R_VALUE                        0
xrs_A_mean                     0
xrs_A_min                      0
xrs_A_max                      0
xrs_B_mean                     0
xrs_B_min                      0
xrs_B_max                      0
Radio Flux 10.7cm              0
Sunspot Number                 0
Sunspot Area (10^6 Hemis.)     0
New Regions                   91
Flares: C                     91
Flares: M                     91
Flares: X 

In [185]:
# Cleaning NaNs by interpolation

# Those 354 days with NaNs in flares are lost in my datasets. I decided to keep them as NaN andcreate a missing column.
df_flares = df_flares.dropna()

nan_count = df_flares.isna().sum()
print(nan_count)

print(df_flares.head())

DATE                          0
USFLUX                        0
MEANGAM                       0
MEANGBT                       0
MEANGBZ                       0
MEANGBH                       0
MEANJZD                       0
TOTUSJZ                       0
MEANALP                       0
MEANJZH                       0
TOTUSJH                       0
ABSNJZH                       0
SAVNCPP                       0
MEANPOT                       0
TOTPOT                        0
MEANSHR                       0
SHRGT45                       0
R_VALUE                       0
xrs_A_mean                    0
xrs_A_min                     0
xrs_A_max                     0
xrs_B_mean                    0
xrs_B_min                     0
xrs_B_max                     0
Radio Flux 10.7cm             0
Sunspot Number                0
Sunspot Area (10^6 Hemis.)    0
New Regions                   0
Flares: C                     0
Flares: M                     0
Flares: X                     0
flare_to

In [186]:
# Check for infinities
print(np.isinf(df_flares.select_dtypes(include=[np.number])).sum().sum())  # Count of inf/-inf
print(np.isnan(df_flares.select_dtypes(include=[np.number])).sum().sum())  # Count of NaN

0
0


In [187]:
# save final datafreme
df_flares.to_csv('./Data/data_flares_del.csv', index=False)